# Plot open interest over time

* Query trades and create long/short OI charts


In [32]:
import os
from copy import deepcopy
from datetime import datetime
import pandas as pd
import numpy as np
import sqlite3
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots

load_dotenv()

pio.renderers.default = 'notebook'

In [33]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")


## Historical funding rate

In [34]:
df_trades = pd.read_sql_query('SELECT * FROM trades', con)


In [14]:
def add_position_cols(df):
    open_positions = {}

    positions = []
    long_position_size = []
    short_position_size = []
    for _, row in df.iterrows():
        # Carry over open positions from previous row
        if _ > 0:
            open_positions = prev_open_positions
        # Update position size for current account
        account = row['account']
        size = row['positionSize']
        if account in open_positions:
            open_positions[account] = size
        else:
            open_positions[account] = size
        # Calculate total long and short position size at current record
        long_position_size.append(sum(
            [open_positions[account] for account in open_positions if open_positions[account] > 0]))
        short_position_size.append(sum([-1*open_positions[account]
                                for account in open_positions if open_positions[account] < 0]))
        positions.append(open_positions)
        # Save current open positions for next iteration
        prev_open_positions = open_positions.copy()

    df['open_positions'] = positions
    df['long_oi'] = long_position_size
    df['short_oi'] = short_position_size

    df['long_oi_pct'] = df.apply(
        lambda x: x['long_oi'] / (x['long_oi'] + x['short_oi']) if (x['long_oi'] + x['short_oi']) > 0 else .5, axis=1)

    df['short_oi_pct'] = df.apply(
        lambda x: x['short_oi'] / (x['long_oi'] + x['short_oi']) if (x['long_oi'] + x['short_oi']) > 0 else .5, axis=1)
    return df




In [17]:
assets = df_trades['asset'].unique().tolist()

df_final = pd.DataFrame()
for asset in assets:
    df_asset = df_trades[df_trades['asset'] == asset].sort_values('timestamp').reset_index()
    df_asset = add_position_cols(df_asset)
    df_final = pd.concat([df_final, df_asset], axis=0)

df_final

,index,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed,open_positions,long_oi,short_oi,long_oi_pct,short_oi_pct
0,5906,0x1c066d56fa5a32ede7e95a02b3e0141dff5783dc4a2a...,0xf68d2bfcecd7895bba05a7451dd09a1749026454,1675912856,OP,sOPPERP,DelayedOffchain,2.592819,198.000000,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1,4.999791,0.000000,771.307300,771.307300,0,{'0xf68d2bfcecd7895bba05a7451dd09a1749026454':...,771.307300,0.0000,1.000000,0.000000
1,36489,0xadf3ef572f51d7917a1764c4d92c7f1460cf4378e155...,0xf68d2bfcecd7895bba05a7451dd09a1749026454,1675916573,OP,sOPPERP,DelayedOffchain,2.608770,205.302667,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1,4.012163,12.302602,-771.307300,0.000000,1,{'0xf68d2bfcecd7895bba05a7451dd09a1749026454':...,0.000000,0.0000,0.500000,0.500000
2,5895,0x1bf90f96239c85fac23eb2d7e2533ccfa402ee79c78c...,0xf68d2bfcecd7895bba05a7451dd09a1749026454,1675944723,OP,sOPPERP,DelayedOffchain,2.678635,201.290504,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x2,3.518687,0.000000,-377.975200,-377.975200,0,{'0xf68d2bfcecd7895bba05a7451dd09a1749026454':...,0.000000,377.9752,0.000000,1.000000
3,29705,0x8dbb519cce52b9c81b57bfdf22fa37a7632d4bb8f2a5...,0xf68d2bfcecd7895bba05a7451dd09a1749026454,1675948183,OP,sOPPERP,DelayedOffchain,2.683027,196.112031,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x2,3.014118,-1.659890,377.975200,0.000000,1,{'0xf68d2bfcecd7895bba05a7451dd09a1749026454':...,0.000000,0.0000,0.500000,0.500000
4,43416,0xcf8f11f879393f65cc092c6d37fa958bd83c70177d2b...,0xf68d2bfcecd7895bba05a7451dd09a1749026454,1675948350,OP,sOPPERP,DelayedOffchain,2.686111,193.097914,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x3,4.925772,0.000000,726.148300,726.148300,0,{'0xf68d2bfcecd7895bba05a7451dd09a1749026454':...,726.148300,0.0000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,18719,0x58661c35b716f1abe1c716a119b516318bcff0689099...,0x6fe0e8bdac611e9df3334cdae26062ba621c434e,1679479988,GBP,sGBPPERP,DelayedOffchain,1.228939,2956.120262,0x1dad8808d8ac58a0df912adc4b215ca3b93d6c49-0x20,13.925084,28.734491,-59863.567300,0.000000,1,{'0x043e7c673f2bd9c62e69921395bfd1f97acc0a78':...,4508.185300,41680.2127,0.097604,0.902396
65,53994,0xbb64e23e62cdf8d470173ecc80c39822c6bae758dd63...,0xad63e4d4be2229b080d20311c1402a2e45cf7e75,1679652208,GBP,sGBPPERP,DelayedOffchain,1.220846,7498.000000,0x1dad8808d8ac58a0df912adc4b215ca3b93d6c49-0x21,38.625389,0.000000,-150000.000000,-150000.000000,0,{'0x043e7c673f2bd9c62e69921395bfd1f97acc0a78':...,4508.185300,191680.2127,0.022979,0.977021
66,54012,0xf9e925cbc2db7f96b875b070f76d93684584071c89c3...,0xad63e4d4be2229b080d20311c1402a2e45cf7e75,1679653411,GBP,sGBPPERP,DelayedOffchain,1.221811,7314.240925,0x1dad8808d8ac58a0df912adc4b215ca3b93d6c49-0x21,9.466731,-58.946118,61112.000000,-88888.000000,0,{'0x043e7c673f2bd9c62e69921395bfd1f97acc0a78':...,4508.185300,130568.2127,0.033375,0.966625
67,54030,0x2fe94e579c45f021f1a660450c33bf1ae7d465baf1cf...,0xd42d654d065ff9560c823f2d71411535d5e81132,1679654819,GBP,sGBPPERP,DelayedOffchain,1.220885,449.021975,0x1dad8808d8ac58a0df912adc4b215ca3b93d6c49-0x1d,2.584744,0.000000,4789.512918,9297.698218,0,{'0x043e7c673f2bd9c62e69921395bfd1f97acc0a78':...,9297.698218,130568.2127,0.066476,0.933524


In [31]:
asset = 'BNB'

fig = px.area(
    df_final[df_final['asset'] == asset],
    x='timestamp',
    y=['short_oi_pct', 'long_oi_pct'],
    color_discrete_sequence=['red', 'green']
)

fig.show()

In [41]:
for ind, asset in enumerate(assets):
    asset_fig = px.area(
        df_final[df_final['asset'] == asset],
        x='timestamp',
        y=['short_oi_pct', 'long_oi_pct'],
        color_discrete_sequence=['red', 'green'],
        title=f"{asset} Open Interest"
    )

    asset_fig.show()
